# Monitoring App

#### Este notebook será usado para las pruebas del proyecto Monitoring App Ecadem

In [1]:
from apscheduler.schedulers.background import BackgroundScheduler
from datetime import datetime
import time
import requests as req
import pandas as pd
import json

In [2]:
import MySQLdb

HOST='us-east.connect.psdb.cloud'
USERNAME='g16cfqqc7npgp4c16oui'
PASSWORD='pscale_pw_QmEuVrfeZWv4AXnOeepFTiSUHdpHh5NZxGLHgDSKcqe'
DATABASE='ecademprod'
SSL_CERT='/etc/ssl/certs/ca-certificates.crt'

connection = MySQLdb.connect(
  host= HOST,
  user= USERNAME,
  passwd= PASSWORD,
  db= DATABASE,
  ssl      = {
    "ca": SSL_CERT
  }
)


cursor = connection.cursor()

In [3]:
CREATION_QUERY = """INSERT INTO ecademprod.Proj_Monitoring_Hist
    (
        idMonitoring,
        lastUpdate,
        time,
        statusCode
    )
    VALUES
    (
        '{}',
        '{}',
        '{}',
        '{}'
    );"""

In [4]:
SELECT_QUERY = """
SELECT 
    id, 
    url, 
    intervalo 
FROM ecademprod.Proj_Monitoring WHERE isActive = 1;"""

df = pd.read_sql(SELECT_QUERY, connection)



In [5]:
def ping_info(id_, url, CREATION_QUERY):

    HOST='us-east.connect.psdb.cloud'
    USERNAME='g16cfqqc7npgp4c16oui'
    PASSWORD='pscale_pw_QmEuVrfeZWv4AXnOeepFTiSUHdpHh5NZxGLHgDSKcqe'
    DATABASE='ecademprod'
    SSL_CERT='/etc/ssl/certs/ca-certificates.crt'

    connection = MySQLdb.connect(
    host= HOST,
    user= USERNAME,
    passwd= PASSWORD,
    db= DATABASE,
    ssl      = {
        "ca": SSL_CERT
    }
    )


    cursor = connection.cursor()

    start = time.time()
    r = req.get(url)
    end = time.time() - start

    results = {
        'id': id_, 
        'last_update': datetime.now(),
        'time': round(end, 2), #round to 2 decimals
        'status': r.status_code 
        
    }

    query = CREATION_QUERY.format(
        results['id'],
        results['last_update'],
        results['time'],
        results['status']
    )

    cursor.execute(query)
    connection.commit()
    connection.close()

In [6]:
to_monitoring = json.loads(df.to_json(orient='records'))

[{'args': [1,
   'https://ecademcontent.ecademservices.com/',
   "INSERT INTO ecademprod.Proj_Monitoring_Hist\n    (\n        idMonitoring,\n        lastUpdate,\n        time,\n        statusCode\n    )\n    VALUES\n    (\n        '{}',\n        '{}',\n        '{}',\n        '{}'\n    );"],
  'interval': 1},
 {'args': [2,
   'https://genpass.ecademservices.com/',
   "INSERT INTO ecademprod.Proj_Monitoring_Hist\n    (\n        idMonitoring,\n        lastUpdate,\n        time,\n        statusCode\n    )\n    VALUES\n    (\n        '{}',\n        '{}',\n        '{}',\n        '{}'\n    );"],
  'interval': 1},
 {'args': [3,
   'https://ecadem.co/',
   "INSERT INTO ecademprod.Proj_Monitoring_Hist\n    (\n        idMonitoring,\n        lastUpdate,\n        time,\n        statusCode\n    )\n    VALUES\n    (\n        '{}',\n        '{}',\n        '{}',\n        '{}'\n    );"],
  'interval': 1},
 {'args': [4,
   'https://genpass.ecadem.co/',
   "INSERT INTO ecademprod.Proj_Monitoring_Hist\n   

In [16]:
ping_info(
    to_monitoring[0]['id'],
    to_monitoring[0]['url'],
    CREATION_QUERY
)


In [ ]:
to_monitoring[1]['']

In [17]:



scheduler = BackgroundScheduler()
for site in to_monitoring:
    scheduler.add_job(
        ping_info,
        'interval',
        args=[
            site['id'],
            site['url'],
            CREATION_QUERY
        ],
        seconds=15 #site['intervalo']
    )
scheduler.start()

In [18]:

scheduler.shutdown()


    
    

## job get data

In [11]:
MIN_INTERVAL_QUERY = """SELECT
	MIN(intervalo) AS minInterval
FROM
	ecademprod.Proj_Monitoring"""


GET_DATA_QUERY = """SELECT 
	LAST_REC.idMonitoring,
	INFO.url,
	INFO.photoUrl,
	LAST_REC.Recent,
	HIST.`time`,
	HIST.statusCode
FROM
	(
	SELECT
			idMonitoring,
			MAX(lastUpdate) AS Recent
	FROM
			ecademprod.Proj_Monitoring_Hist
	GROUP BY
		idMonitoring
) AS LAST_REC
JOIN ecademprod.Proj_Monitoring_Hist AS HIST
ON
	LAST_REC.idMonitoring = HIST.idMonitoring
	AND LAST_REC.Recent = HIST.lastUpdate 
JOIN ecademprod.Proj_Monitoring AS INFO
ON INFO.id = LAST_REC.idMonitoring"""

In [12]:
interval = pd.read_sql(MIN_INTERVAL_QUERY, connection).values[0][0]
interval

1

In [42]:
data = pd.read_sql(GET_DATA_QUERY, connection) 
resp = json.loads( 
    data.to_json(orient='records', date_format='iso')
    )

In [19]:
with open('./data.json', 'w') as outfile:
    json.dump(resp, outfile)

In [17]:
resp

[{'idMonitoring': 4,
  'url': 'https://genpass.ecadem.co/',
  'photoUrl': None,
  'Recent': '2023-04-25T07:44:20.000Z',
  'time': 0.24,
  'statusCode': 200},
 {'idMonitoring': 2,
  'url': 'https://genpass.ecademservices.com/',
  'photoUrl': None,
  'Recent': '2023-04-25T07:44:20.000Z',
  'time': 0.26,
  'statusCode': 200},
 {'idMonitoring': 1,
  'url': 'https://ecademcontent.ecademservices.com/',
  'photoUrl': None,
  'Recent': '2023-04-25T07:44:20.000Z',
  'time': 0.48,
  'statusCode': 200},
 {'idMonitoring': 3,
  'url': 'https://ecadem.co/',
  'photoUrl': None,
  'Recent': '2023-04-25T07:44:21.000Z',
  'time': 0.85,
  'statusCode': 200}]

In [20]:
# resp[0]['Recent'] = 
datetime.strptime(resp[0]['Recent'], '%Y-%m-%dT%H:%M:%S.%fZ')
# resp[0]['Recent'].isoformat()

datetime.datetime(2023, 4, 25, 7, 44, 20)

In [43]:
for i in resp:
    # Color
    if i['statusCode'] == 200:
        i['statusCode'] = "#21f104"
    else:
        i['statusCode'] = "#c60000"

    # Date
    i['Recent'] = datetime.strptime(str(i['Recent']), '%Y-%m-%dT%H:%M:%S.%fZ')

    # url
    try:
        i['url'] = re.search('https?://([\w.-]+\.[\w.-]+).*', i['url']).group(1)
    except:
        print(i['url'])
        
    # url photo
    if i['photoUrl'] == None:
        i['photoUrl'] = 'https://i.imgur.com/kJ00MaT.png'

    
    
    


In [23]:
import re

url = 'https://ecadem.co/'
domain = re.search('https?://([\w.-]+\.[\w.-]+).*', url).group(1)

print(domain)

ecadem.co


In [44]:
resp

[{'idMonitoring': 4,
  'url': 'genpass.ecadem.co',
  'photoUrl': 'https://i.imgur.com/kJ00MaT.png',
  'Recent': datetime.datetime(2023, 4, 25, 7, 44, 20),
  'time': 0.24,
  'statusCode': '#21f104'},
 {'idMonitoring': 2,
  'url': 'genpass.ecademservices.com',
  'photoUrl': 'https://i.imgur.com/kJ00MaT.png',
  'Recent': datetime.datetime(2023, 4, 25, 7, 44, 20),
  'time': 0.26,
  'statusCode': '#21f104'},
 {'idMonitoring': 1,
  'url': 'ecademcontent.ecademservices.com',
  'photoUrl': 'https://i.imgur.com/kJ00MaT.png',
  'Recent': datetime.datetime(2023, 4, 25, 7, 44, 20),
  'time': 0.48,
  'statusCode': '#21f104'},
 {'idMonitoring': 3,
  'url': 'ecadem.co',
  'photoUrl': 'https://i.imgur.com/kJ00MaT.png',
  'Recent': datetime.datetime(2023, 4, 25, 7, 44, 21),
  'time': 0.85,
  'statusCode': '#21f104'}]